# テキスト分類のファインチューニング
IMDbデータセット上でDistilBERTをファインチューニングして、映画レビューが肯定的か否定的かを判断するモデルを作ります。

公式Notebook：https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/text_classification.ipynb

## load dataset

In [ ]:
# !pip install transformers datasets

In [6]:
from datasets import load_dataset

imdb = load_dataset("imdb")

Found cached dataset imdb (/Users/hajime/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
imdb["test"][0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

- text: 映画レビューのテキスト
- label: 否定的なレビューの場合は0、肯定的なレビューの場合は1

## preprocess

1. モデルに先立って、事前学習モデルとセットのトークナイザーをインスタンス化します。
2. テキストをトークン化し、DistilBERTの最大入力長以下になるようにシーケンスを切り詰めるpreprocess_function()という関数を作成します。tructionがその役目です。データセット全体に対して前処理関数を適用するために、map関数を使用します。 `batched=True`を設定すると、map関数を高速化することができます。
3. DataCollatorWithPadding を使用して、データをサンプルするバッチを作成します。また、バッチ内の最も長い要素の長さに合わせてテキストを動的にパディングし、データの長さをそろえます。トークナイザー関数の中で padding=True を設定してテキストを詰めることもできますが、動的なパディングの方がより効率的です。

In [14]:
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding

# 1
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# 2
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_imdb = imdb.map(preprocess_function, batched=True)

# 3
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Loading cached processed dataset at /Users/hajime/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-282ea8cadface71d.arrow
Loading cached processed dataset at /Users/hajime/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-0044d531500dfb33.arrow


  0%|          | 0/50 [00:00<?, ?ba/s]

## Train
1. TrainingArgumentsに学習用ハイパーパラメータを定義する。
2. TrainingArgumentsをmodel、dataset、tokenizer、data_collatorと一緒にTrainerに渡す。
3. train()を呼び出し、モデルのファインチューニングを行う。

In [15]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results_seq",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier

  0%|          | 0/7815 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


# トークン単位の分類（NER）のファインチューニング
WNUT 17データセットでDistilBERTをファインチューニングして、新しいエンティティを検出するモデルを作ります。

公式Notebook: https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/token_classification.ipynb

## load dataset

In [ ]:
from datasets import load_dataset

wnut = load_dataset("wnut_17")

In [ ]:
wnut["train"][0]

ner_tagsの各数値は、エンティティを表します。数字をラベル名に変換すると、詳細が表示されます。

In [ ]:
label_list = wnut["train"].features[f"ner_tags"].feature.names
label_list

ner_tagは、企業、場所、人などの実体を表します。各ner_tagの前に付く文字は、そのエンティティのトークン位置を示します。

- B-はエンティティの先頭を示す。
- I-は、トークンが同じエンティティに含まれることを示す（たとえば、StateトークンはEmpire State Buildingのようなエンティティの一部）。
- 0は、そのトークンがどのエンティティにも対応しないことを示す。

## preprocess

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# 入力はすでに単語に分割されているので、単語をサブワードにトークン化するために is_split_into_words=True を設定する。
tokenized_input = tokenizer(wnut["train"][0]["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

特殊トークン[CLS]と[SEP]の追加とサブワードによる分割より、入力とラベルの間にミスマッチが生じます。1つのラベルに対応する1つの単語が2つのサブワードに分割されることがあります。以下の方法で、トークンとラベルを再調整する必要があります。

1. word_ids メソッドですべてのトークンを対応する単語に対応付ける。
2. 特殊なトークンである [CLS] と [SEP] にラベル -100 を割り当て、PyTorch の損失関数がそれらを無視するようにする。
3. 与えられた単語の最初のトークンのみにラベルを付ける。同じ単語の他のサブトークンに-100を割り当てる。

以下は、トークンとラベルを再調整し、シーケンスをDistilBERTの最大入力長以下に切り詰める関数を作成する方法です。

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # 1
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # 2
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # 3
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

お決まりのmap関数とdata_collator

In [ ]:
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Train

In [ ]:
training_args = TrainingArguments(
    output_dir="./results_ner",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

# Question Answering
Question Answeringは、与えられた質問に対して回答を返すものです。問題設定は２パターンあります。

- 抽出型: 与えられた文脈から答えを抽出する。
- 生成型: 質問に正しく答えるために、文脈から答えを生成します。

今回は抽出型のためにSQuADデータセット上でDistilBERTをファインチューニングしてモデルを作ります。

公式Notebook: https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/question_answering.ipynb


## load dataset

In [ ]:
from datasets import load_dataset

squad = load_dataset("squad")

In [ ]:
squad["train"][0]

answersフィールドは、答えの開始位置と答えのテキストを含むdictです。

## preprocess
データセットの中のいくつかの例は、モデルの最大入力長を超える非常に長いコンテキストを持っていることがあります。

1. truncation="only_second "とすることで、文脈だけを切り詰めることができます。
2. 次に、return_offset_mapping=Trueを設定して、答えの開始位置と終了位置を元のコンテキストにマッピングします。
3. マッピングが完了したら、答えの開始と終了のトークンを見つけることができます。sequence_ids メソッドを使用して、オフセットのどの部分が質問に対応し、どの部分がコンテキストに対応するかを見つけます。

以下は、答えの開始と終了のトークンを切り捨ててコンテキストにマッピングする関数を作成する方法です。

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # コンテキストの開始と終了を検索する
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # 答えが文脈の中に完全にない場合は、ラベル(0, 0)を追加する
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # それ以外の場合は、開始と終了のトークンの位置を追加する
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

お決まりのmap関数。今回は不要なカラムを削除します。

お決まりのdata_collatorも。

In [ ]:
from transformers import DefaultDataCollator

tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

data_collator = DefaultDataCollator(tokenizer=tokenizer)

## Train

In [ ]:
training_args = TrainingArguments(
    output_dir="./results_qa",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

# Language Model
ELI5データセットのr/askscienceサブセット上で、Causal LMのDistilGPT2とMasked LMのDistilRoBERTaをファインチューニングして新しいモデルを作ります。

公式Notebook：https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/language_modeling.ipynb

In [ ]:
from datasets import load_dataset

eli5 = load_dataset("eli5", split="train_asks[:5000]")

# trainとtestに分割
eli5 = eli5.train_test_split(test_size=0.2)

# データセットの前処理を行う際、textサブフィールドを別のカラムに取り出す必要があります。
eli5 = eli5.flatten()


In [ ]:
eli5["train"][0]

## preprocess
各サブフィールドは、answerという接頭辞で示されるように、独立したカラムになっています。answers.textはリストであることに注意してください。各文を個別にトークン化する代わりに、リスト内の文をつなげてからまとめてトークン化します。

リストを文字列に変換し、DistilGPT2の最大入力長より長くならないようにシーケンスを切り捨てる前処理関数を作成する方法を示します。

In [ ]:
from transformers import AutoTokenizer

# Causal LM
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

# Masked LM
# tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]], truncation=True)

お決まりのmap関数。今回も不要なカラムを削除しています。num_proc で処理数を増やして高速化で来ます。

今回は、長いデータから切り捨てられたテキストを取り込むためのもう一つ前処理の関数を適用する必要があります。
- すべてのテキストを連結する。
- 連結されたテキストを block_size で定義される小さなチャンクに分割する。

In [ ]:
tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=eli5["train"].column_names,
)

block_size = 128


def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

DataCollatorForLanguageModelingを使用して、データサンプルのバッチを作成します。

In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Masked LM
# データを反復処理するたびにランダムにトークンをマスクする mlm_probability を設定します。
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

## Train

In [ ]:
from transformers import AutoModelForCausalLM, AutoModelForMaskedLM, TrainingArguments, Trainer

# Causal LM
model = AutoModelForCausalLM.from_pretrained("distilgpt2")

# Masked LM
# model = AutoModelForMaskedLM.from_pretrained("distilroberta-base")

training_args = TrainingArguments(
    output_dir="./results_clm",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

## Evaluation
Perplexityは、予測単語候補の数です。小さい方が良い。後半の講義で取り上げます。

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

# Summarization

要約もQAと同じく問題設定が二つあります。
- 抽出型：文書から最も関連性の高い情報を抽出する。
- 生成型：最も関連性の高い情報をとらえた新しいテキストを生成する。

BillSumデータセットのカリフォルニア州法案サブセットでT5をファインチューニングして、生成型の要約を行うモデルを作ります。

公式Notebook：https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/summarization.ipynb


## load dataset

In [ ]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")
# trainとtestに分割
billsum = billsum.train_test_split(test_size=0.2)

In [ ]:
billsum["train"][0]

## preprocess

1. 入力に`"summarize: "`を付加し、T5がこれが要約タスクであることを認識できるようにする。複数の自然言語処理タスクが可能なモデルの中には、特定のタスクのためのプロンプトを必要とするものがある。
2. ラベルをトークン化する際に text_target 引数を使用する。
3. シーケンスを切り捨てて、max_length パラメータで設定された最大長以下にする。

後はお決まりのmap関数と、data_collator.

In [ ]:
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq

tokenizer = AutoTokenizer.from_pretrained("t5-small")

prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_billsum = billsum.map(preprocess_function, batched=True)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

## Train

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

training_args = Seq2SeqTrainingArguments(
    output_dir="./results_sum",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()